In [17]:
# based on Hu et al. (2023) - Interpretable Machine Learning based on Functional ANOVA Framework: Algorithms and Comparisons
# generate 20 variables from a multivariate gaussian distribution with mean 0, variance 1 and equal correlation 0.5 between all pairs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(0)
n = 2000
p = 20
mean = np.zeros(p)
cov = np.ones((p,p)) * 0.5 + np.eye(p) * 0.5 # creates a covariance matrix with 0.5 correlation between all pairs and variance 1 on the diagonal
data_20 = np.random.multivariate_normal(mean, cov, n)
# truncate in interval [-2.5, 2.5]
data_20[data_20 > 2.5] = 2.5
data_20[data_20 < -2.5] = -2.5
df_20 = pd.DataFrame(data_20, columns=range(1, 21))

# generate 10 more variables
n = 2000
p = 10
mean = np.zeros(p)
cov = np.ones((p,p)) * 0.5 + np.eye(p) * 0.5
data_10 = np.random.multivariate_normal(mean, cov, n)
# truncate in interval [-2.5, 2.5]
data_10[data_10 > 2.5] = 2.5
data_10[data_10 < -2.5] = -2.5
# create a dataframe, starting the columns from 20
df_10 = pd.DataFrame(data_10, columns=range(21, 31))

# combine the two datasets
X = pd.concat([df_20, df_10], axis=1)
X

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
0,-1.002388,-1.153842,-1.689717,-1.591207,-2.198869,-1.218741,-1.021193,-1.921186,-0.830534,-1.293703,...,1.934008,1.626153,2.254498,1.379249,0.838128,0.182266,2.122122,1.074466,1.667598,1.305231
1,2.300292,2.054176,0.654455,0.432740,2.500000,2.185950,2.500000,0.999045,2.500000,1.834949,...,-1.112134,-0.785854,-0.067828,-0.272249,-0.005966,0.023397,-0.177265,0.444406,0.714334,0.412917
2,-0.218931,-0.219003,0.392896,1.590406,1.637891,1.054559,0.204761,0.296159,0.534972,-0.110044,...,-2.366773,-0.132697,-0.284187,-0.111117,-0.116168,-0.541848,-0.493288,-1.261793,-0.876211,-0.660513
3,0.239439,-0.555784,0.970629,-0.278846,1.426454,0.763015,0.647401,0.035322,0.748140,0.640731,...,-1.283346,-2.500000,-1.161551,-1.619342,-1.065846,-2.016316,-0.587726,-0.609948,-0.640994,-1.441232
4,1.465083,1.411186,-0.031690,-0.429587,-0.592471,0.218205,1.374044,1.033817,1.143093,0.585513,...,-0.031910,0.023663,-0.247120,0.510007,-0.335710,-0.641410,-1.357639,-0.269618,-0.936559,-0.903473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-0.780668,0.185577,0.741944,1.467525,-0.152930,0.269904,0.387794,0.368488,1.569149,0.182011,...,1.616366,0.558311,1.120472,-0.241217,0.547155,0.433571,0.206292,0.843564,-1.094648,0.028489
1996,-1.229072,-0.630826,-1.730775,-1.487419,-1.021030,-1.871527,-1.476404,-2.018067,-0.412612,-0.301728,...,-0.732515,0.220307,-1.406922,-0.235610,0.670852,-0.260360,0.001478,-0.329981,-0.987782,-0.635334
1997,0.901566,0.828429,1.804260,1.146357,0.476770,0.564665,0.680933,0.140176,1.482964,1.541697,...,0.019390,0.679264,0.530607,2.137807,1.655895,1.133427,1.697084,1.350135,0.913814,1.781905
1998,0.143354,0.082367,0.267493,-0.549626,0.124704,-0.852645,1.717698,0.135873,-0.245046,0.403029,...,1.813848,0.358661,0.377175,0.388133,-0.032060,0.842295,0.548184,0.584648,0.403399,0.464973


In [18]:
# calculate the following function based on the current instance of X:
# y = \sum_{j=1}^5x_j + 0.5*\sum_{j=6}^8x_j^2+\sum_{j=9}^{10} x_j I(x_j>0) + x_1x_2 + x_1x_3 + x_2x_3 + 0.5x_1x_2x_3 + x_4x_5 + x_4x_6 + x_5x_6 + 0.5I(x_4>0)x_5x_6

def g(x):
    y = np.sum(x[:,0:5], axis=1) + 0.5*np.sum(x[:,5:8]**2, axis=1) + np.sum(x[:,8:10] * (x[:,8:10] > 0), axis=1) + x[:,0]*x[:,1] + x[:,0]*x[:,2] + x[:,1]*x[:,2] + 0.5*x[:,0]*x[:,1]*x[:,2] + x[:,3]*x[:,4] + x[:,3]*x[:,5] + x[:,4]*x[:,5] + 0.5*(x[:,3]>0)*x[:,4]*x[:,5]
    # add some noise N(0, 0.5^2)
    y += np.random.normal(0, 0.5, y.shape)
    return y

In [19]:
# generate the target variable y
y = g(X.values)
y

array([ 6.87553703, 37.21503678, 10.96846665, ..., 14.05859031,
        2.17410318,  2.58094523])

In [20]:
# save the data
X.to_csv('../datasets/synth_Hu4.csv', index=False)
pd.DataFrame(y).to_csv('../datasets/synth_Hu4_y.csv', index=False)